In [5]:
import pandas as pd
from clickhouse_driver import Client
host="192.168.1.101"
port=9099
user="sjjc"
password="taosan506"
database="dm"
send_receive_timeout=10
client = Client(host=host, port=port, user=user, password=password, database=database,
                        send_receive_timeout=send_receive_timeout)

In [6]:
# 用户基本信息处理，选出所有的列来
sql = """
select uid from dm.pri_cust_base_info order by uid
"""
a = client.execute(sql)

In [7]:
name_attribute = ['uid']
data=pd.DataFrame(columns=name_attribute,data=a)
data.set_index('uid',inplace=True)

In [8]:
# 持有贷记卡张数
data_len = len(data)

In [9]:
def exec_eql_and_concat(data, sql, attr_list, default_na):
    # 执行sql
    data_list = client.execute(sql)
    # 转为df
    new_data_df = pd.DataFrame(columns=attr_list,data=data_list)
    # 按uid拼接
    new_data_df.set_index("uid", inplace=True)
    data1 = data.join(new_data_df,how='left',sort=False)
    # 补全空缺值
    data1.fillna(default_na,inplace=True)
    # 返回
    return data1

In [67]:
select_djk_num = """
select uid, count(card_no) from dm.dm_v_as_djk_info group by uid order by uid;
"""
data = exec_eql_and_concat(data,select_djk_num,['uid','djk_num'],{'djk_num':0})

In [68]:
normal_djk_sql = """
select uid, count(card_no) from dm.dm_v_as_djk_info where card_sts_name='正常' group by uid order by uid
"""
data = exec_eql_and_concat(data,normal_djk_sql,['uid','normal_djk_num'],{'normal_djk_num':0})
data.describe()

,djk_num,normal_djk_num
count,376186.000000,376186.000000
mean,0.051358,0.029443
std,0.233359,0.170345
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,7.000000,4.000000


In [69]:
# 用户逾期/止付卡片
abnormal_djk_num_sql = """
select uid, count(card_no) from dm.dm_v_as_djk_info where card_sts_name='银行止付卡片' or card_sts_name='因逾期停息停滞纳金' or card_sts_name='因逾期而停用卡片' group by uid order by uid
"""
data = exec_eql_and_concat(data,abnormal_djk_num_sql,['uid','abnormal_djk_num'],{'abnormal_djk_num':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num
count,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103
std,0.233359,0.170345,0.045807
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000


In [70]:
bal_duebill_sql = """select uid, min(bal) as bal_duebill from dm.dm_v_tr_duebill_mx group by uid order by uid;"""
data = exec_eql_and_concat(data,bal_duebill_sql,['uid','bal_duebill'],{'bal_duebill':0})
data.describe()


,djk_num,normal_djk_num,abnormal_djk_num
count,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103
std,0.233359,0.170345,0.045807
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000


In [71]:
normal_bal_duebill_sql = """select uid, min(norm_bal) norm_bal from dm.dm_v_tr_duebill_mx group by uid order by uid;"""
data = exec_eql_and_concat(data,normal_bal_duebill_sql,['uid','normal_bal_duebill'],{'normal_bal_duebill':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num
count,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103
std,0.233359,0.170345,0.045807
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000


In [72]:
online_tran_duebill_sql = """select uid,  count(*) as online_tran_duebill from dm.dm_v_tr_duebill_mx where loan_channel = '手机银行' or loan_channel = '网上银行' group by uid  order by uid;
"""
data = exec_eql_and_concat(data,online_tran_duebill_sql,['uid','online_tran_duebill'],{'online_tran_duebill':0})
data.describe()



,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill
count,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185
std,0.233359,0.170345,0.045807,3.990931
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000


In [73]:
offline_tran_duebill_sql = """select uid,  count(*) as offline_tran_duebill from dm.dm_v_tr_duebill_mx where loan_channel = '柜面' group by uid order by uid;
"""
data = exec_eql_and_concat(data,offline_tran_duebill_sql,['uid','offline_tran_duebill'],{'offline_tran_duebill':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304
std,0.233359,0.170345,0.045807,3.990931,0.589840
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000


In [74]:
has_car_etc_sql ="""select uid, 1 as has_car_etc from dm_v_tr_etc_mx group by uid order by uid;"""
data = exec_eql_and_concat(data,has_car_etc_sql,['uid','has_car_etc'],{'has_car_etc':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000


In [75]:
has_personal_bank_grwy_sql = """select uid, 1 as has_personal_bank_grwy from dm.dm_v_tr_grwy_mx group by uid order by uid;
 """
data = exec_eql_and_concat(data,has_personal_bank_grwy_sql,['uid','has_personal_bank_grwy'],{'has_personal_bank_grwy':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000


In [76]:
MCA_times_grwy_sql = """select uid, count(*) as MCA_times_grwy from dm.dm_v_tr_grwy_mx  where sys_type = 'MCA' group by uid order by uid;
"""
data = exec_eql_and_concat(data,MCA_times_grwy_sql,['uid','MCA_times_grwy'],{'MCA_times_grwy':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy,MCA_times_grwy
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011,0.000098
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261,0.039770
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000,21.000000


In [77]:
Core_times_grwy_sql = """select uid, count(*) as Core_times_grwy from dm.dm_v_tr_grwy_mx  where sys_type = 'Core' group by uid order by uid;
"""
data = exec_eql_and_concat(data,Core_times_grwy_sql,['uid','Core_times_grwy'],{'Core_times_grwy':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy,MCA_times_grwy,Core_times_grwy
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011,0.000098,0.000080
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261,0.039770,0.040334
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000,21.000000,24.000000


In [78]:
has_social_security_gzdf = """select uid, 1 as has_social_security_gzdf from dm.dm_v_tr_gzdf_mx where is_secu_card = '1' group by uid order by uid;
"""
data = exec_eql_and_concat(data,has_social_security_gzdf,['uid','has_social_security_gzdf'],{'has_social_security_gzdf':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy,MCA_times_grwy,Core_times_grwy,has_social_security_gzdf
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011,0.000098,0.000080,0.230958
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261,0.039770,0.040334,0.421446
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000,21.000000,24.000000,1.000000


In [79]:
bal_huanb = """select uid, min(bal) as bal_huanb from dm.dm_v_tr_huanb_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,bal_huanb,['uid','bal_huanb'],{'bal_huanb':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy,MCA_times_grwy,Core_times_grwy,has_social_security_gzdf
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011,0.000098,0.000080,0.230958
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261,0.039770,0.040334,0.421446
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000,21.000000,24.000000,1.000000


In [80]:
max_pay_times_huanb_sql = """select uid, max(pay_term) as max_pay_times_huanb from dm.dm_v_tr_huanb_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,max_pay_times_huanb_sql,['uid','max_pay_times_huanb'],{'max_pay_times_huanb':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy,MCA_times_grwy,Core_times_grwy,has_social_security_gzdf,max_pay_times_huanb
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011,0.000098,0.000080,0.230958,0.425712
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261,0.039770,0.040334,0.421446,5.428617
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000,21.000000,24.000000,1.000000,175.000000


In [81]:
tran_amt_huanb = """select uid, sum(tran_amt) as tran_amt_huanb from dm.dm_v_tr_huanb_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,tran_amt_huanb,['uid','tran_amt_huanb'],{'tran_amt_huanb':0})
data.describe()
tran_times_huanb = """select uid, count(*) as tran_times_huanb from dm.dm_v_tr_huanb_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,tran_times_huanb,['uid','tran_times_huanb'],{'tran_times_huanb':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy,MCA_times_grwy,Core_times_grwy,has_social_security_gzdf,max_pay_times_huanb,tran_times_huanb
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011,0.000098,0.000080,0.230958,0.425712,0.347248
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261,0.039770,0.040334,0.421446,5.428617,3.805687
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000,21.000000,24.000000,1.000000,175.000000,825.000000


In [82]:
tran_amt_duebill = """select uid, sum(buss_amt) as tran_amt_duebill from dm.dm_v_tr_duebill_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,tran_amt_duebill,['uid','tran_amt_duebill'],{'tran_amt_duebill':0})
tran_times_duebill = """select uid, count(*) as tran_times_duebill from dm.dm_v_tr_duebill_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,tran_times_duebill,['uid','tran_times_duebill'],{'tran_times_duebill':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy,MCA_times_grwy,Core_times_grwy,has_social_security_gzdf,max_pay_times_huanb,tran_times_huanb,tran_times_duebill
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011,0.000098,0.000080,0.230958,0.425712,0.347248,0.393927
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261,0.039770,0.040334,0.421446,5.428617,3.805687,4.319643
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000,21.000000,24.000000,1.000000,175.000000,825.000000,1471.000000


In [83]:
tran_amt_etc = """select uid, sum(tran_amt_fen) as tran_amt_etc from dm.dm_v_tr_etc_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,tran_amt_etc,['uid','tran_amt_etc'],{'tran_amt_etc':0})
tran_times_etc = """select uid, count(*) as tran_times_etc from dm.dm_v_tr_etc_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,tran_times_etc,['uid','tran_times_etc'],{'tran_times_etc':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy,MCA_times_grwy,Core_times_grwy,has_social_security_gzdf,max_pay_times_huanb,tran_times_huanb,tran_times_duebill,tran_times_etc
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011,0.000098,0.000080,0.230958,0.425712,0.347248,0.393927,0.978245
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261,0.039770,0.040334,0.421446,5.428617,3.805687,4.319643,34.949180
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000,21.000000,24.000000,1.000000,175.000000,825.000000,1471.000000,17384.000000


In [84]:
tran_amt_grwy = """select uid, sum(tran_amt) as tran_amt_grwy from dm.dm_v_tr_grwy_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,tran_amt_grwy,['uid','tran_amt_grwy'],{'tran_amt_grwy':0})
tran_times_grwy = """select uid, count(*) as tran_times_grwy from dm.dm_v_tr_grwy_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,tran_times_grwy,['uid','tran_times_grwy'],{'tran_times_grwy':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy,MCA_times_grwy,Core_times_grwy,has_social_security_gzdf,max_pay_times_huanb,tran_times_huanb,tran_times_duebill,tran_times_etc,tran_times_grwy
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011,0.000098,0.000080,0.230958,0.425712,0.347248,0.393927,0.978245,0.000178
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261,0.039770,0.040334,0.421446,5.428617,3.805687,4.319643,34.949180,0.073550
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000,21.000000,24.000000,1.000000,175.000000,825.000000,1471.000000,17384.000000,36.000000


In [85]:
tran_amt_gzdf = """select uid, sum(tran_amt) as tran_amt_gzdf from dm.dm_v_tr_gzdf_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,tran_amt_gzdf,['uid','tran_amt_gzdf'],{'tran_amt_gzdf':0.0})
tran_times_gzdf = """select uid, count(*) as tran_times_gzdf from dm.dm_v_tr_gzdf_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,tran_times_gzdf,['uid','tran_times_gzdf'],{'tran_times_gzdf':0})
data.describe()

,djk_num,normal_djk_num,abnormal_djk_num,online_tran_duebill,offline_tran_duebill,has_car_etc,has_personal_bank_grwy,MCA_times_grwy,Core_times_grwy,has_social_security_gzdf,max_pay_times_huanb,tran_times_huanb,tran_times_duebill,tran_times_etc,tran_times_grwy,tran_times_gzdf
count,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000,376186.000000
mean,0.051358,0.029443,0.002103,0.245185,0.052304,0.025022,0.000011,0.000098,0.000080,0.230958,0.425712,0.347248,0.393927,0.978245,0.000178,1.554359
std,0.233359,0.170345,0.045807,3.990931,0.589840,0.156193,0.003261,0.039770,0.040334,0.421446,5.428617,3.805687,4.319643,34.949180,0.073550,4.542923
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,7.000000,4.000000,1.000000,1471.000000,62.000000,1.000000,1.000000,21.000000,24.000000,1.000000,175.000000,825.000000,1471.000000,17384.000000,36.000000,424.000000


In [86]:
data.to_csv('data.csv',index=True, encoding='utf-8')

In [10]:
# 表中就一个月份
csv_data = pd.read_csv('result.csv',encoding='utf-8')
month_shop_times_sql = """select uid, count(*) as month_shop_avg from dm.dm_v_tr_shop_mx group by uid order by uid;
"""
csv_data = exec_eql_and_concat(csv_data,month_shop_times_sql,['uid','month_shop_avg'],{'month_shop_times':0})
